# Ridge Regression 

Following are the objectives 

* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression
* Use matplotlib to visualize polynomial regressions
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression, this time with L2 penalty
* Use matplotlib to visualize polynomial regressions under L2 regularization
* Choose best L2 penalty using cross-validation.
* Assess the final fit using test data.


# Fire up graphlab create

In [1]:
import graphlab

# Polynomial regression, revisited

Writing a function to produce an SFrame with columns containing the powers of a given input.

In [2]:
def polynomial_sframe(feature, degree):
    # assume that degree >= 1
    # initialize the SFrame:
    poly_sframe = graphlab.SFrame()
    # and set poly_sframe['power_1'] equal to the passed feature
    poly_sframe['power_1'] = feature

    # first check if degree > 1
    if degree > 1:
        # then loop over the remaining degrees:
        # range usually starts at 0 and stops at the endpoint-1. We want it to start at 2 and stop at degree
        for power in range(2, degree+1): 
            # first we'll give the column a name:
            name = 'power_' + str(power)
            # then assign poly_sframe[name] to the appropriate power of feature
            poly_sframe[name] = poly_sframe['power_1'].apply(lambda x: x**power)

    return poly_sframe

Using matplotlib to visualize what a polynomial regression looks like on the house data.

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to vipulsatone@gmail.com and will expire on June 26, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\VIPULS~1\AppData\Local\Temp\graphlab_server_1484248849.log.0


Using the sqft_living variable. For plotting purposes (connecting the dots), sorting by the values of sqft_living. For houses with identical square footage, we break the tie by their prices.

In [5]:
sales = sales.sort(['sqft_living','price'])

Generating polynomial features up to degree 15 using `polynomial_sframe()` and fit a model with these features. When fitting the model, use an L2 penalty of `1e-5`:

In [6]:
l2_small_penalty = 1e-5

In [7]:
import math
import random
import numpy


def print_coefficients(model):    
    # Get the degree of the polynomial
    deg = len(model.coefficients['value'])-1

    # Get learned parameters as a list
    w = list(model.coefficients['value'])

    # Numpy has a nifty function to print out polynomials in a pretty way
    # (We'll use it, but it needs the parameters in the reverse order)
    print 'Learned polynomial for degree ' + str(deg) + ':'
    w.reverse()
    print numpy.poly1d(w)

poly1_data = polynomial_sframe(sales['sqft_living'], 1)
poly1_data['price'] = sales['price'] # add price to the data since it's the target
print poly1_data
model = graphlab.linear_regression.create(poly1_data, target='price', l2_penalty=1e-5,l1_penalty=0.,validation_set=None,verbose=False)
print_coefficients(model)
print model.coefficients['value']

+---------+----------+
| power_1 |  price   |
+---------+----------+
|  290.0  | 142000.0 |
|  370.0  | 276000.0 |
|  380.0  | 245000.0 |
|  384.0  | 265000.0 |
|  390.0  | 228000.0 |
|  390.0  | 245000.0 |
|  410.0  | 325000.0 |
|  420.0  | 229050.0 |
|  420.0  | 280000.0 |
|  430.0  | 80000.0  |
+---------+----------+
[21613 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
Learned polynomial for degree 1:
 
280.6 x - 4.358e+04
[-43580.73867200711, 280.62356583789614]


# first running the model without L2 penalty and later with L2 penalty.

Spliting the data into split the sales data into four subsets of roughly equal size and call them `set_1`, `set_2`, `set_3`, and `set_4`. Use `.random_split` function with `seed=0`. 

In [8]:
data3 = polynomial_sframe(sales['sqft_living'], 15)

features_list = data3.column_names()
data3['price'] = sales['price']


(semi_split1, semi_split2) = data3.random_split(.5,seed=0)
(set_1, set_2) = semi_split1.random_split(0.5, seed=0)
(set_3, set_4) = semi_split2.random_split(0.5, seed=0)

Fitting a 15th degree polynomial on `set_1`, `set_2`, `set_3`, and `set_4`, using 'sqft_living' to predict prices. Printing the weightsof each model.


In [9]:

model1 = graphlab.linear_regression.create(set_1, target='price', l2_penalty=1e-5,l1_penalty=0.,features=features_list,validation_set=None,verbose=False)
print_coefficients(model1)
print model1.coefficients['value']

Learned polynomial for degree 15:
            15             14           13             12
-2.338e-53 x  + 1.527e-49 x  + 3.8e-45 x  + 2.851e-41 x 
              11             10             9             8
 - 1.627e-37 x  - 6.838e-33 x  - 6.928e-29 x + 1.593e-25 x
              7             6             5             4             3
 + 1.069e-20 x + 5.975e-17 x - 3.798e-13 x - 1.529e-08 x + 0.0001415 x
           2
 - 0.3973 x + 585.9 x + 9306
[9306.463978137821, 585.8658134702486, -0.3973058847241758, 0.00014147089482517821, -1.529459743939567e-08, -3.797565260616573e-13, 5.974818473203499e-17, 1.0688850597902126e-20, 1.5934405234889325e-25, -6.928349551499124e-29, -6.838133680447517e-33, -1.6268620390826704e-37, 2.8511878428745745e-41, 3.799981469166738e-45, 1.5265261705823444e-49, -2.338073102523865e-53]


In [10]:
model2 = graphlab.linear_regression.create(set_2, target='price', l2_penalty=1e-5,l1_penalty=0.,features=features_list,validation_set=None,verbose=False)
print_coefficients(model2)
print model2.coefficients['value']

Learned polynomial for degree 15:
           15             14             13             12
1.301e-50 x  - 1.004e-46 x  - 1.024e-42 x  - 9.413e-40 x 
              11             10             9             8
 + 6.517e-35 x  + 6.218e-31 x  - 2.713e-29 x - 4.597e-23 x
              7             6             5             4             3
 - 2.064e-19 x + 2.512e-15 x + 6.843e-12 x - 1.152e-07 x + 0.0004388 x
           2
 - 0.7678 x + 783.5 x - 2.512e+04
[-25115.905986942584, 783.4938025082208, -0.7677593001731524, 0.0004387663619337906, -1.151691611523392e-07, 6.842811487074161e-12, 2.5119522463968243e-15, -2.064406243436433e-19, -4.5967305882838754e-23, -2.7127734249181614e-29, 6.218185050566659e-31, 6.517413117435824e-35, -9.413162759868363e-40, -1.0242136312864189e-42, -1.0039109975339404e-46, 1.3011336684824306e-50]


In [11]:
model3 = graphlab.linear_regression.create(set_3, target='price', l2_penalty=1e-5,l1_penalty=0.,features=features_list,validation_set=None,verbose=False)
print_coefficients(model3)
print model3.coefficients['value']

Learned polynomial for degree 15:
            15             14             13             12
-2.858e-51 x  + 3.556e-47 x  + 2.426e-43 x  - 1.066e-39 x 
              11            10             9             8
 - 2.968e-35 x  - 1.72e-31 x  + 1.243e-27 x + 2.571e-23 x
              7             6             5             4             3
 + 4.088e-20 x - 2.082e-15 x - 2.261e-12 x + 1.154e-07 x - 0.0005283 x
          2
 + 1.029 x - 759.3 x + 4.624e+05
[462426.56573111366, -759.2518428541024, 1.028670047297077, -0.0005282645273860696, 1.1542290838547994e-07, -2.2609594806249738e-12, -2.082142875706699e-15, 4.087704757092605e-20, 2.5707913290013165e-23, 1.2431126519598316e-27, -1.720258349392118e-31, -2.9676107131469384e-35, -1.0657489049865715e-39, 2.426356214581897e-43, 3.5559876473049924e-47, -2.8577746872292375e-51]


In [12]:
model4 = graphlab.linear_regression.create(set_4, target='price', l2_penalty=1e-5,l1_penalty=0.,features=features_list,validation_set=None,verbose=False)
print_coefficients(model4)
print model4.coefficients['value']

Learned polynomial for degree 15:
           15             14             13             12
4.395e-50 x  - 6.029e-46 x  - 1.799e-42 x  + 2.334e-38 x 
              11             10             9             8
 + 2.399e-34 x  + 1.602e-31 x  - 1.324e-26 x - 7.999e-23 x
              7             6             5             4             3
 + 4.215e-19 x + 4.818e-15 x - 2.202e-11 x - 6.383e-08 x + 0.0005553 x
          2
 - 1.225 x + 1248 x - 1.702e+05
[-170240.0347907073, 1247.5903508836507, -1.224609126404328, 0.0005552546267870285, -6.382623619291666e-08, -2.2021599647495865e-11, 4.818346975937941e-15, 4.214616324799642e-19, -7.998807490509938e-23, -1.3236590770565815e-26, 1.6019779713868089e-31, 2.3990433732615665e-34, 2.3335450576522715e-38, -1.798740558949416e-42, -6.028626828938645e-46, 4.39472672530924e-50]


The four curves should differ from one another a lot.

# Ridge regression comes to rescue setting L2 penalty

In [13]:
model1 = graphlab.linear_regression.create(set_1, target='price', l2_penalty=1e5,l1_penalty=0.,validation_set=None,verbose=False)
print_coefficients(model1)

Learned polynomial for degree 15:
           15             14             13             12
3.601e-58 x  + 5.845e-54 x  + 9.392e-50 x  + 1.498e-45 x 
              11             10             9             8
 + 2.382e-41 x  + 3.789e-37 x  + 6.066e-33 x + 9.861e-29 x
             7             6             5            4             3
 + 1.65e-24 x + 2.896e-20 x + 5.422e-16 x + 1.06e-11 x + 1.749e-07 x
             2
 + 0.001274 x + 2.587 x + 5.303e+05


In [14]:
model2 = graphlab.linear_regression.create(set_2, target='price', l2_penalty=1e5,l1_penalty=0.,validation_set=None,verbose=False)
print_coefficients(model2)

Learned polynomial for degree 15:
           15             14             13             12
4.715e-55 x  + 4.427e-51 x  + 4.183e-47 x  + 3.985e-43 x 
              11             10             9             8
 + 3.842e-39 x  + 3.767e-35 x  + 3.777e-31 x + 3.901e-27 x
             7             6             5             4             3
 + 4.16e-23 x + 4.531e-19 x + 4.808e-15 x + 4.435e-11 x + 2.931e-07 x
             2
 + 0.001131 x + 2.045 x + 5.192e+05


In [15]:
model3 = graphlab.linear_regression.create(set_3, target='price', l2_penalty=1e5,l1_penalty=0.,validation_set=None,verbose=False)
print_coefficients(model3)

Learned polynomial for degree 15:
           15             14             13             12
1.917e-55 x  + 1.912e-51 x  + 1.911e-47 x  + 1.915e-43 x 
              11             10             9             8
 + 1.927e-39 x  + 1.954e-35 x  + 2.004e-31 x + 2.093e-27 x
              7             6             5             4             3
 + 2.247e-23 x + 2.501e-19 x + 2.876e-15 x + 3.209e-11 x + 2.776e-07 x
             2
 + 0.001259 x + 2.269 x + 5.229e+05


In [16]:
model3 = graphlab.linear_regression.create(set_3, target='price', l2_penalty=1e5,l1_penalty=0.,validation_set=None,verbose=False)
print_coefficients(model3)

Learned polynomial for degree 15:
           15             14             13             12
1.917e-55 x  + 1.912e-51 x  + 1.911e-47 x  + 1.915e-43 x 
              11             10             9             8
 + 1.927e-39 x  + 1.954e-35 x  + 2.004e-31 x + 2.093e-27 x
              7             6             5             4             3
 + 2.247e-23 x + 2.501e-19 x + 2.876e-15 x + 3.209e-11 x + 2.776e-07 x
             2
 + 0.001259 x + 2.269 x + 5.229e+05


These curves should vary a lot less, now that you applied a high degree of regularization.

# Selecting an L2 penalty via cross-validation

In [17]:
(train_valid, test) = sales.random_split(.9, seed=1)
train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)

In [18]:
n = len(train_valid_shuffled)
k = 10 # 10-fold cross-validation

for i in xrange(k):
    start = (n*i)/k
    end = (n*(i+1))/k-1
    print i, (start, end)

0 (0, 1938)
1 (1939, 3878)
2 (3879, 5817)
3 (5818, 7757)
4 (7758, 9697)
5 (9698, 11636)
6 (11637, 13576)
7 (13577, 15515)
8 (15516, 17455)
9 (17456, 19395)


Extracting the fourth segment (segment 3) and assign it to a variable called `validation4`.

In [19]:
validation4 =train_valid_shuffled[5818:7758]

Extracting the remainder of the data after *excluding* fourth segment (segment 3) and assign the subset to `train4`.

In [20]:

first = train_valid_shuffled[0:5818]
last = train_valid_shuffled[7758:19396]
train4 = first.append(last)
print train4

+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 2780400035 | 2014-05-05 00:00:00+00:00 | 665000.0 |   4.0    |    2.5    |
| 1703050500 | 2015-03-21 00:00:00+00:00 | 645000.0 |   3.0    |    2.5    |
| 5700002325 | 2014-06-05 00:00:00+00:00 | 640000.0 |   3.0    |    1.75   |
| 0475000510 | 2014-11-18 00:00:00+00:00 | 594000.0 |   3.0    |    1.0    |
| 0844001052 | 2015-01-28 00:00:00+00:00 | 365000.0 |   4.0    |    2.5    |
| 2781280290 | 2015-04-27 00:00:00+00:00 | 305000.0 |   3.0    |    2.5    |
| 2214800630 | 2014-11-05 00:00:00+00:00 | 239950.0 |   3.0    |    2.25   |
| 2114700540 | 2014-10-21 00:00:00+00:00 | 366000.0 |   3.0    |    2.5    |
| 2596400050 | 2014-07-30 00:00:00+00:00 | 375000.0 |   3.0    |    1.0    |
| 4140900050 | 2015-01-26 00:00:00+00:00 | 440000.0 |   4.0    |    1.75   |

Now we are ready to implement k-fold cross-validation. Write a function that computes k validation errors by designating each of the k segments as the validation set. It accepts as parameters (i) `k`, (ii) `l2_penalty`, (iii) dataframe, (iv) name of output column (e.g. `price`) and (v) list of feature names. The function returns the average validation error using k segments as validation sets.

* For each i in [0, 1, ..., k-1]:
  * Compute starting and ending indices of segment i and call 'start' and 'end'
  * Form validation set by taking a slice (start:end+1) from the data.
  * Form training set by appending slice (end+1:n) to the end of slice (0:start).
  * Train a linear model using training set just formed, with a given l2_penalty
  * Compute validation error using validation set just formed

In [21]:
def k_fold_cross_validation(k, l2_penalty, data, output_name, features_list):
    
    n = len(data)
    error = numpy.array([0,0,0,0,0,0,0,0,0,0],dtype=float)
    #print error
    #print 'HHHHHHHHHHHHHHHHHH',l2_penalty
    for i in range (1 , k):
        start = (n*i)/k
        end = (n*(i+1))/k-1
        validation_set = data[start:end+1]
        first = data[0:start]
        last = data[end+1:n]
        training_set = first.append(last)
        lm = graphlab.linear_regression.create(training_set, target= output_name ,features = features_list, l2_penalty=l2_penalty,l1_penalty=0.,validation_set=None,verbose=False)
        t = validation_set['price']-lm.predict(validation_set)
        tt1 = t*t
        ttt1 =tt1.sum()
        error[i] = ttt1
    print error.sum(), l2_penalty

running above function in a loop and finding L2 penalty for which error is minimum.

In [22]:

data1 = polynomial_sframe(train_valid_shuffled['sqft_living'], 15)

features_list = data1.column_names()
data1['price'] = train_valid_shuffled['price']
hh = numpy.logspace(1,7, num=13)
for l2_penalty in (hh):
    k_fold_cross_validation(10, l2_penalty, data1, 'price', features_list)

4.80842968848e+15 10.0
2.76518424411e+15 31.6227766017
1.49939980354e+15 100.0
1.11132084056e+15 316.227766017
1.10189111923e+15 1000.0
1.12630178179e+15 3162.27766017
1.24247584723e+15 10000.0
1.55945423603e+15 31622.7766017
2.09469823461e+15 100000.0
2.3023226434e+15 316227.766017
2.34637481233e+15 1000000.0
2.38258587216e+15 3162277.66017
2.40151475761e+15 10000000.0


After finding the best L2 penalty training the model on entire dataset using this penalty and printing RSS.

In [23]:
train2 = polynomial_sframe(train_valid['sqft_living'], 15)
features_list = train2.column_names()
train2['price'] = train_valid['price']
lmf = graphlab.linear_regression.create(train2, target='price',features = features_list, l2_penalty=1000,l1_penalty=0.,validation_set=None,verbose=False)
t = test['price']-lmf.predict(test)
tt1 = t*t
ttt1 =tt1.sum()
print ttt1

2.52897427447e+14
